In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *
from fastai.metrics import *
from fastai.model import *
from fastai.dataset import *
from fastai.torch_imports import *
from fastai.io import *

import torch.nn as nn

ModuleNotFoundError: No module named 'PIL'

In [3]:
df = pd.read_csv('data/MNIST/train.csv')
df.shape

NameError: name 'pd' is not defined

In [4]:
# .values convert to np array
y = df['label'].values  
X = df.drop(['label'], axis=1).values

NameError: name 'df' is not defined

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((33600, 784), (8400, 784), (33600,), (8400,))

In [10]:
mean = X_train.mean()
std = X_train.std()

X_train = (X_train-mean) / std
X_valid = (X_valid-mean) / std

In [11]:
net = nn.Sequential(
    nn.Linear(28*28, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 10),
    nn.LogSoftmax()
).cuda()

In [12]:
md = ImageClassifierData.from_arrays('tmp/', (X_train, y_train), (X_valid, y_valid))

In [13]:
loss = nn.NLLLoss()
metrics = [accuracy]
opt = optim.SGD(net.parameters(), 1e-1, momentum=0.9, weight_decay=1e-3)

In [14]:
set_lrs(opt, 1e-2)

In [15]:
fit(net, md, n_epochs=5, crit=loss, opt=opt, metrics=metrics)

epoch      trn_loss   val_loss   accuracy                     
    0      0.224409   0.217919   0.936786  
    1      0.146333   0.150967   0.955357                     
    2      0.097729   0.133188   0.95881                      
    3      0.077969   0.120039   0.963095                      
    4      0.069907   0.107223   0.96631                       



[array([0.10722]), 0.9663095238095238]

In [16]:
preds = predict(net, md.val_dl)
preds = preds.argmax(1)
np.mean(preds == y_valid)

0.9663095238095238

In [17]:
df_test = pd.read_csv('data/MNIST/test.csv')

In [18]:
X_test = df_test.values

In [19]:
preds_test = predict(net, X_test)
preds_test = preds_test.argmax(1)

TypeError: forward() takes 2 positional arguments but 784 were given